In [1]:
import torch
torch.set_num_threads(2)
# from huggingface_hub import hf_hub_download
# from safetensors import


In [2]:
from anydoor_refiners.model import AnyDoor, AnydoorAutoencoder
from anydoor_refiners.dinov2 import DINOv2Encoder
from anydoor_refiners.unet import UNet
from anydoor_refiners.controlnet import ControlNet

lda = AnydoorAutoencoder(device="cuda", dtype=torch.float16)
encoder = DINOv2Encoder(device="cuda",dtype=torch.float16)
unet = UNet(4,device="cuda",dtype=torch.float16)
controlnet = ControlNet(4,device="cuda",dtype=torch.float16)
anydoor = AnyDoor(
    unet=unet,
    lda=lda,
    control_model=controlnet,
    object_encoder=encoder,
    device="cuda",
    dtype=torch.float16,
)

In [ ]:
# anydoor.unet.load_from_safetensors("../ckpt/unet_refiners.safetensors")
# anydoor.control_model.load_from_safetensors("../ckpt/controlnet.safetensors")
# anydoor.object_encoder.load_from_safetensors("../ckpt/encoder.safetensors")
# anydoor.lda.load_from_safetensors(
#     hf_hub_download(
#         repo_id="refiners/sd15.autoencoder",
#         filename="model.safetensors",
#     )
# )

In [8]:
# hyperparameters
seed = 42
uncod_scale = 5.0
num_inference_steps = 50
anydoor.set_inference_steps(num_inference_steps, first_step=0)

In [26]:
# inserted_object_img : torch.Tensor = ...
# background_control_img : torch.Tensor = ...

# object, control = preprocessing(inserted_object_img,background_control_img)
object_image = torch.randn(1, 3, 224, 224,device="cuda",dtype=torch.float16)
control_image = torch.randn(1, 4, 256, 256,device="cuda",dtype=torch.float16)

In [ ]:
from refiners.fluxion.utils import manual_seed, no_grad
from tqdm import tqdm

with no_grad():  
    manual_seed(seed)
    object_embedding = anydoor.object_encoder(object_image)
    negative_object_embedding = anydoor.object_encoder(torch.zeros((object_image.shape[0], 3, 224, 224),device="cuda",dtype=torch.float16))
    x = anydoor.init_latents((256, 256))

    # diffusion denoising process
    for step in tqdm(anydoor.steps):
        x = anydoor.forward(
            x,
            step=step,
            control_background_image= control_image,
            object_embedding= object_embedding,
            negative_object_embedding= negative_object_embedding,
            condition_scale= 5.0
        )
    predicted_image = anydoor.lda.latents_to_image(x)



In [28]:
for i,t in enumerate(control_dino):
    print(t.shape, control[i].shape)

torch.Size([1, 320, 32, 32]) torch.Size([1, 320, 32, 32])
torch.Size([1, 320, 32, 32]) torch.Size([1, 320, 32, 32])
torch.Size([1, 320, 32, 32]) torch.Size([1, 320, 32, 32])
torch.Size([1, 320, 16, 16]) torch.Size([1, 320, 16, 16])
torch.Size([1, 640, 16, 16]) torch.Size([1, 640, 16, 16])
torch.Size([1, 640, 16, 16]) torch.Size([1, 640, 16, 16])
torch.Size([1, 640, 8, 8]) torch.Size([1, 640, 8, 8])
torch.Size([1, 1280, 8, 8]) torch.Size([1, 1280, 8, 8])
torch.Size([1, 1280, 8, 8]) torch.Size([1, 1280, 8, 8])
torch.Size([1, 1280, 4, 4]) torch.Size([1, 1280, 4, 4])
torch.Size([1, 1280, 4, 4]) torch.Size([1, 1280, 4, 4])
torch.Size([1, 1280, 4, 4]) torch.Size([1, 1280, 4, 4])
torch.Size([1, 1280, 4, 4]) torch.Size([1, 1280, 4, 4])


In [7]:
with torch.no_grad():       
    x = torch.randn(1, 4, 32, 32, dtype=torch.float16, device="cuda")
    timestep = torch.full((1,), 1, dtype=torch.long, device="cuda")
    object_embedding = torch.randn(1, 1, 1024, dtype=torch.float16, device="cuda")
    control = torch.load("tests/tensors/control_features.pt",weights_only=True)
    control = [t.to("cuda").to(torch.float16) for t in control]
    anydoor.unet.set_timestep(timestep)
    anydoor.unet.set_dinov2_object_embedding(object_embedding)
    anydoor.unet.set_control_residuals(control)
    y = anydoor.unet(x)

In [11]:
with torch.no_grad():
    timestep = anydoor.solver.timesteps[1].unsqueeze(dim=0)

    anydoor.set_unet_context(
            timestep=timestep,
            object_embedding=object_embedding,
            control_features=control,
        )
    anydoor.unet(x)

In [15]:
len(control)

13

In [16]:
timestep

tensor([961], device='cuda:0')